# Measurement setup
![](./setup.png)

# Preliminary

In [2]:
%matplotlib
import sys, os, re
import numpy as np
import matplotlib.pyplot as plt
import datetime
import visa

# Add path
sys.path.append('../Instrument Driver')
sys.path.append('../Helper')

# import custom modules
import myplots_py3

# # import instrument driver
import nidaqmx.stream_readers
import Lakeshore332

class Params(object):
    """ Parameter holder. Parameters are defined here."""
    def __init__(self, date_today=None):
        self.date_today = datetime.datetime.today().strftime('%m/%d/%Y')
    
class DAQ(object):
    """
    DAQ factory with 2 analog inputs
    Args: sample_num, sample_rate
    """
    def __init__(self, sample_num=2000, sample_rate=20e3):
        self.sample_num = sample_num
        self.sample_rate = sample_rate
        self.DAQ_analog_inputs = np.zeros((2, sample_num), dtype=np.float64)
        
        # create a task
        self.task = nidaqmx.task.Task() 
        self.ai = nidaqmx.stream_readers.AnalogMultiChannelReader(self.task.in_stream) # create ai
        
        # set analog inputs
        self.task.ai_channels.add_ai_voltage_chan("Dev1/ai0") # for I 
        self.task.ai_channels.add_ai_voltage_chan("Dev1/ai1") # for V
        
        # set external trigger
        self.task.triggers.start_trigger.cfg_dig_edge_start_trig('/Dev1/PFI0')

        # set sampling rate and number of samples
        self.task.timing.cfg_samp_clk_timing(self.sample_rate, samps_per_chan=self.sample_num) # set sampling rate per channel
                      
    def get_DAQ_analog_inputs(self):        
        """ Read AI0 and AI1 """
        self.ai.read_many_sample(self.DAQ_analog_inputs, self.sample_num)
        return self.DAQ_analog_inputs
    
    @staticmethod
    def calculate_IV_from_preamp_voltages(AI0, AI1, params):
        """ Calculate I and V from preamp volages, i.e., analog input 0 and 1."""
        I = AI0 / params.preamp_gain_for_I/params.standard_resistance
        V = AI1/ params.preamp_gain_for_V

        return I, V

def create_header(params):
    """  Create a header using parameters"""
    header = ''
    for k,v in vars(params).items():
        header += str(k) + ': ' + str(v) + '\n'
    header += 'Current(A), Voltage(V)'
    
    return header

def save_data(data, params):
    """ Save data in ascii file.
        Data file consists of a header and numeric data.
        Each line in the header starts with '#'.
        Args: 
            data: numpy array
            params: instance of Params class
        No return
        """
    
    with open(params.full_file_path,'wb') as f:
        header = create_header(params)
        np.savetxt(f, data, fmt='%.3e', header=header)

def get_full_file_name(params):
    """ Create a full file name with file extension 'dat'.
        File name structure:
        ID number_Sample ID_Exp name.dat. ex) 1_NbTi_VI.dat
    """
    full_file_name = get_file_ID_number(params) + '_' + params.sample_ID + '_' + params.exp_name + '.dat'
    
    return full_file_name

def get_file_ID_number(params):
    """ Find ID number used to create a full file name. 
        Under base path, search for all ID numbers of files and decide
        what ID to be used next
        Args: 
            params: instance of Params class
        Return: next ID number in str to be used
        """
    ID_numbers = []
    for (root,dirs,files) in os.walk(params.base_path, topdown=True):
        
        for file in files:
            matchobj = re.match(r'^\d+', file)
            if matchobj:
                ID_numbers.append(int(matchobj.group()))    
    if ID_numbers:
        return str(np.max(ID_numbers) + 1)
    else:
        return str(1)

def coroutine(func):
    def start(*arg, **kwarg):
        g = func(*arg, **kwarg)
        next(g)
        return g
    return start
 
@coroutine
def moving_average_generator(num_avg):
    """ Generator for moving average of VI curve.
        data_queue acts as a container.
        Args:
            num_avg: number of average
    """
    if num_avg==1:
        next_data = None
        while True:
            next_data = yield next_data
    
    for x in range(num_avg):
        
        if x == 0:
            next_data = yield
            data_queue = next_data
            next_data = yield next_data
        else:
            data_queue = np.row_stack((data_queue, next_data))
            next_data = yield np.mean(data_queue, axis=0)
        
    while True:        
        np.delete(data_queue, 0, axis=0)
        data_queue = np.row_stack((data_queue, next_data))
        next_data = yield np.mean(data_queue, axis=0)
        
# class VI_Meas(object):
#     """
#     Do VI measurement
#     """

#     def __init__(self, standard_resistance, preamp_gain_for_I, preamp_gain_for_V, sample_num, sample_rate ):
#         self.standard_resistance = standard_resistance
#         self.preamp_gain_for_I = preamp_gain_for_I
#         self.preamp_gain_for_V = preamp_gain_for_V
#         self.sample_num = sample_num
#         self.sample_rate = sample_rate

#         self._configure_DAQ()

#         self.DAQ_AI0, self.DAQ_AI1 = np.array([]), np.array([])

#     def _configure_DAQ(self):
#         """
#             Configure NI DAQ; two analog inputs, trigger, sampling rate and number
#             of samples.
#         """
#         # create a task
#         self.task = nidaqmx.task.Task() 
#         self.ai = nidaqmx.stream_readers.AnalogMultiChannelReader(self.task.in_stream) # create ai

#         # set analog inputs
#         self.task.ai_channels.add_ai_voltage_chan("Dev1/ai0") # for I 
#         self.task.ai_channels.add_ai_voltage_chan("Dev1/ai1") # for V

#         # set external trigger
#         self.task.triggers.start_trigger.cfg_dig_edge_start_trig('/Dev1/PFI0')

#         # set sampling rate and number of samples
#         self.task.timing.cfg_samp_clk_timing(self.sample_rate, samps_per_chan=self.sample_num) # set sampling rate per channel

#     def getVI(self):
#         """
#             Take data from two analog inputs in DAQ.
#             Return:
#                     Actual sample voltage and current through it.
#         """
#         DAQ_analog_inputs = np.zeros((2,sample_num), dtype=np.float64)
#         self.ai.read_many_sample(DAQ_analog_inputs, self.sample_num) # take data
#         self.DAQ_AI0 = DAQ_analog_inputs[0,:]
#         self.DAQ_AI1 = DAQ_analog_inputs[1,:]
        
#         I = AI0 / self.preamp_gain_for_I / self.standard_resistance,  # convert preamp voltage to actual V and I
#         V = AI1 / self.preamp_gain_for_V

#         return I, V, self.DAQ_AI0, self.DAQ_AI1

Using matplotlib backend: Qt5Agg


In [130]:
import os
p=Params()
p.base_path = './'
p.sample_ID ='NbTi'
p.exp_name='test'

In [3]:
@coroutine
def test(num_avg):
    for x in range(num_avg):
        
        if x == 0:
            next_data = yield
            data_queue = next_data
            next_data = yield next_data
        else:
            data_queue = np.row_stack((data_queue, next_data))
            next_data = yield np.mean(data_queue, axis=0)
            
g = test(5)
x = np.linspace(0,1,11)


In [4]:
g.send(x*2)

array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ])

In [135]:
DAQplotter = myplots_py3.plotDAQ(xlabel1='Time (s)', ylabel1='Preamp Volt.\n for I (V)'
                                , xlabel2='Time (s)', ylabel2='Preamp Volt.\n for V (V)'
                                , xlabel3='Current, I (A)', ylabel3='Voltage, V (V)'
                                , title='VI Meas', figsize=(10,10))
sample_num = 2000   # number of data points to take per trigger
sampling_rate = 20e3  # sampling rate

timedata = np.linspace(0, sample_num/sampling_rate, sample_num
                      )  # CHECK THIS
I = np.sin(2*np.pi*11*timedata*4) 
V = np.sin(2*np.pi*11*timedata*4) + np.random.random(sample_num)*0.1
DAQplotter.update(timedata,I,V, I,V, I, 100)

# Take VI continuously, no save

In [3]:
############################################################################
# Set parameters
params = Params()

params.standard_resistance = 10e3    # Ohm, standard resistor
params.preamp_gain_for_I = 1     # Preamp gain for I
params.preamp_gain_for_V = 10   # Preamp gain for V
params.sample_num = 2000   # number of data points to take per trigger
params.sampling_rate = 10e3  # sampling rate
params.num_moving_avg = 1    # number of moving average
#############################################################################

# Create VI_meas object
# VI = VI_meas(standard_resistance
#              , preamp_gain_for_I
#              , preamp_gain_for_V
#              , sample_num
#              , sampling_rate)
daq = DAQ(params.sample_num, params.sampling_rate)

# Create a plot object
DAQplotter = myplots_py3.plotDAQ(xlabel1='Time (s)', ylabel1='Preamp Volt.\n for I (V)'
                                , xlabel2='Time (s)', ylabel2='Preamp Volt.\n for V (V)'
                                , xlabel3='Current, I (A)', ylabel3='Voltage, V (V)'
                                , title='VI Measurement', figsize=(10,10))

# Take data continuously and plot it
timedata = np.linspace(0, params.sample_num/params.sampling_rate, params.sample_num)  # CHECK THIS
V_num_moving_avg_gen = moving_average_generator(params.num_moving_avg)
I_num_moving_avg_gen = moving_average_generator(params.num_moving_avg)

try:
    while True:
        # acquire data in DAQ
    #     I, V, preamp_output_for_I, preamp_output_for_for_V = VI.getVI()
        preamp_output_for_I, preamp_output_for_V = daq.get_DAQ_analog_inputs()
        I, V = daq.calculate_IV_from_preamp_voltages(preamp_output_for_I
                                                    , preamp_output_for_V
                                                    , params)                            
        # moving average
        V_num_moving_avg = V_num_moving_avg_gen.send(V)
        I_num_moving_avg = I_num_moving_avg_gen.send(I)

        # data post-processing
        z = np.polyfit(I_num_moving_avg, V_num_moving_avg, 1) # do linear fit
        V_fit = z[1] + z[0] * I_num_moving_avg
        fit_resistance = z[0]

        # plot VI data and linear fit
        DAQplotter.update(timedata, preamp_output_for_I, preamp_output_for_V
                          , I_num_moving_avg, V_num_moving_avg, V_fit, fit_resistance)  
except KeyboardInterrupt:
    print('KeybaordInterrupt! Program stopped')

KeybaordInterrupt! Program stopped



# Take VI

In [ ]:
############################################################################
# Set parameters
params = Params()

params.standard_resistance = 10e3    # Ohm, standard resistor
params.preamp_gain_for_I = 1     # Preamp gain for I
params.preamp_gain_for_V = 1e3   # Preamp gain for V
params.sample_num = 2000   # number of data points to take per trigger
params.sampling_rate = 20e3  # sampling rate
params.num_avg = 1    # number of average

params.base_path = ''
params.sample_ID = ''
params.exp_name = 'VI'
############################################################################

params.full_file_name = get_full_file_name(params)
params.full_file_path = os.path.join(params.base_path, params.full_file_name)

# Create DAQ object
daq = DAQ(params.sample_num, params.sampling_rate)

# Create a plot object
title = params.full_file_name.replace('.dat', '')
VIplotter = myplots_py3.plotVI(xlabel='Current, I (A)', ylabel='Voltage, V (V)'
                                , title=title, figsize=(10,10))

# Take data
I_avg, V_avg = np.zeros(params.sample_num), np.zeros(params.sample_num)

for i in range(params.num_avg):
    # acquire data
    preamp_output_for_I, preamp_output_for_V = daq.get_DAQ_analog_inputs()
    I, V = daq.calculate_IV_from_preamp_voltages(preamp_output_for_I
                                                , preamp_output_for_V
                                                , params)                            
    # average
    I_avg = (I_avg * i + I) / (i+1)
    V_avg = (V_avg * i + V) / (i+1)
        
    # plot VI data and linear fit
    VIplotter.update(I_avg, V_avg)

# save data with header and save plot as image.
data = np.column_stack((I_avg, V_avg))
save_data(data, params)

fig_full_file_path = params.full_file_path.replace('.dat','.png')
VIplotter.fig.savefig(fig_full_file_path)

# RT

In [ ]:
############################################################################
# Set parameters
params = Params()

params.standard_resistance = 10e3    # Ohm, standard resistor
params.preamp_gain_for_I = 1     # Preamp gain for I
params.preamp_gain_for_V = 1e3   # Preamp gain for V
params.sample_num = 2000   # number of data points to take per trigger
params.sampling_rate = 20e3  # sampling rate
params.num_avg = 1    # number of average

params.base_path = ''
params.sample_ID = ''
params.exp_name = 'VI'
############################################################################

LS = Lakeshore332.Lakeshore332()

params.full_file_name = get_full_file_name(params)
params.full_file_path = os.path.join(params.base_path, params.full_file_name)

# Create DAQ object
daq = DAQ(params.sample_num, params.sampling_rate)

# Create a plot object
title = params.full_file_name.replace('.dat', '')
VIplotter = myplots_py3.plotVI(xlabel='Temperature (K)'
                               , ylabel='Resistence (Ohm)'
                               , title=title, figsize=(10,10))

# Take data
I_avg, V_avg = np.zeros(params.sample_num), np.zeros(params.sample_num)
temperatures, resistances = [], []

with open(params.full_file_path, 'a') as f:
    header = create_header(params)
    np.savetxt(f, [], header=header )

try:
    while True:
        # measure R
        I_avg, V_avg = np.zeros(params.sample_num), np.zeros(params.sample_num)
        for i in range(params.num_avg):
            # acquire data
            preamp_output_for_I, preamp_output_for_V = daq.get_DAQ_analog_input()
            I, V = daq.calculate_IV_from_preamp_voltages(preamp_output_for_I
                                                        , preamp_output_for_V
                                                        , params)           
            I_avg = np.row_stack((I_avg, I))
            V_avg = np.row_stack((V_avg, V))
        I_avg, V_avg = np.mean(I_avg, axis=0), np.mean(V_avg, axis=0)                                
        z = np.polyfit(I_avg, V_avg, 1) # do linear fit
        resistance = z[0]
        resistances.append(resistance)

        # measure temperature
        temperature = LS.getTemperature('B')
        temperatures.append(temperature)

        # plot VI data and linear fit
        VIplotter.update(temperatures, resistances)

        # save data 
        with open(params.full_file_path,'a') as f:
            data = np.array([temperature, resistance])
            np.savetxt(f, [data], fmt='%.3e',delimiter=',')

except KeyboardInterrupt: # to stop taking data 
    fig_full_file_path = params.full_file_path.replace('.dat','.png')
    VIplotter.fig.savefig(fig_full_file_path)

# Test

In [5]:
%matplotlib
import time

sample_num = 2000
sample_rate = 10000  # 10k is max when two channels are used


data = np.zeros((2,sample_num), dtype=np.float64)
task = nidaqmx.task.Task() # create a task obj in NI-max
task.ai_channels.add_ai_voltage_chan("Dev1/ai0") # for I 
task.ai_channels.add_ai_voltage_chan("Dev1/ai1") # for V

task.triggers.start_trigger.cfg_dig_edge_start_trig('/Dev1/PFI0')
task.timing.cfg_samp_clk_timing(sample_rate, samps_per_chan=sample_num) # set sampling rate per channel
# timingCfg = nidaqmx.task.Timing(task._handle) 
# timingCfg.cfg_samp_clk_timing(sample_rate, samps_per_chan=sample_num) # set sampling rate per channel

ai = nidaqmx.stream_readers.AnalogMultiChannelReader(task.in_stream)

DAQplotter = myplots_py3.plotDAQ(xlabel1='time (s)', ylabel1='Raw V for I (V)',
                              xlabel2='time (s)', ylabel2='Raw V for V (V)',
                              xlabel3='I (A)', ylabel3='V (V)',
                              title='VI Meas', figsize=(10,10))

# Take data continuously and plot it
indexArr = np.arange(sample_num)
#timeArr = np.arange(sample_num) * (1/sample_rate)

for i in range(1000):
    ai.read_many_sample(data, sample_num)
    time.sleep(1)
    Iarr, Varr = data[0,:], data[1,:]
    
    Varr = Varr  #+ np.random.rand(sample_num) * 0.1
    
    # data post-processing
    z = np.polyfit(Iarr, Varr, 1) # do linear fit
    Varr_fit = z[1] + z[0] * Iarr

    # plot VI data and linear fit
    DAQplotter.update(indexArr, Iarr, Varr,Iarr,Varr,Varr_fit, z[0])  

task.close()

Using matplotlib backend: Qt5Agg


KeyboardInterrupt: 

In [38]:
import json
ds = {'sample_rate': 20e3, 'sample_num':1200, 'sub':{'a':10, 'b':20e3}}
header1 = json.dumps(ds)
    
header = 'Test header\n'
header += 'I(A), V(V)'
x = np.linspace(0,1,10)
x = np.column_stack((x,x*0.1))
with open('test.dat', 'wb') as f:
    np.savetxt(f, x, header=header, fmt='%.3e')
    

In [4]:
params = Params()

params.standard_resistance = 10e3    # Ohm, standard resistor
params.preamp_gain_for_I = 1     # Preamp gain for I
params.preamp_gain_for_V = 1e3   # Preamp gain for V
params.sample_num = 2000   # number of data points to take per trigger
params.sampling_rate = 20e3  # sampling rate
params.num_avg = 1    # number of average

params.base_path = ''
params.sample_ID = ''
params.exp_name = 'VI'

In [202]:
create_header(params)
# for k,v  in vars(params).items():
#     print(k,v)

'date_today: 11/09/2020\nstandard_resistance: 10000.0\npreamp_gain_for_I: 1\npreamp_gain_for_V: 1000.0\nsample_num: 2000\nsampling_rate: 20000.0\nnum_avg: 1\nbase_path: \nsample_ID: \nexp_name: VI\nCurrent(A), Voltage(V)'

In [203]:
header = create_header(params)
data = []
with open('./test.dat','wb') as f:
    header = create_header(params)
    np.savetxt(f, data, header=header)


In [2]:
VIplotter = myplots_py3.plotVI(xlabel1='Current, I (A)', ylabel1='Voltage, V (V)'
                                , title='', figsize=(10,10))


In [16]:
with open('testttq.dat', 'wb') as f:
    header = create_header(params)
    np.savetxt(f, [], header=header)

for i in range(10):
    
    with open('testttq.dat', 'a') as f:
        data = np.array([i, i*2])        
        np.savetxt(f, [data], fmt='%.3e',delimiter=',')


In [10]:
data.transpose()

array([4, 8])